## Images as arrays

When you scan a subject with MRI scanner, the images are first stored as **DICOM (Digital Imaging and Communications in Medicine)**.

As you see in the description here (https://en.wikibooks.org/wiki/Neuroimaging_Data_Processing/DICOM), most neuroimaging software tools convert them to a lighter, eaier-to-work-with format, **NIFTI**. So the first step in any neuroimaging processing is to convert DICOM to NIFTI format, using toos such as **dcm2nii**.

But reagardless of the file format, what you have to understand first is that images are simply **numerical arrays** that represent strength of a signal (i.e. intensity level) at a given point in space.

To demonstrate this, we will construct an array using **numpy** package and visualize it with **matplotlib** in python.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os.path as op

In [ ]:
an_array = np.array([[ 0,  0,  0,  0,  0,  0,  0,  0],
                     [ 0,  0,  0,  9, 99, 99, 94,  0],
                     [ 0,  0,  0, 25, 99, 99, 79,  0],
                     [ 0,  0,  0,  0,  0,  0,  0,  0],
                     [ 0,  0,  0, 56, 99, 99, 49,  0],
                     [ 0,  0,  0, 73, 99, 99, 31,  0],
                     [ 0,  0,  0, 91, 99, 99, 13,  0],
                     [ 0,  0,  9, 99, 99, 94,  0,  0],
                     [ 0,  0, 27, 99, 99, 77,  0,  0],
                     [ 0,  0, 45, 99, 99, 59,  0,  0],
                     [ 0,  0, 63, 99, 99, 42,  0,  0],
                     [ 0,  0, 80, 99, 99, 24,  0,  0],
                     [ 0,  1, 96, 99, 99,  6,  0,  0],
                     [ 0, 16, 99, 99, 88,  0,  0,  0],
                     [ 0,  0,  0,  0,  0,  0,  0,  0]])

We can check the size of the array.

In [ ]:
an_array.shape

We can show arrays as images using the plt.imshow command. This is the default output:

In [ ]:
plt.imshow(an_array)
plt.colorbar()

We can change the colormap to gray, but the choice is arbitrary. You can choose any color map you like!

In [ ]:
plt.imshow(an_array, cmap='gray')
plt.colorbar()

## NIFTI image

The above example is a small 2-dimensional array. Neuroimaging data are typically larger, and usually 3-D (anatomical image) or 4-D (3-D image with temporal axis).

The NIFTI image contains three basic components:
 
1. data array: this component is similar to an array above, except that it's 3-D or 4-D.
2. header: this component stores various meta-data about the image, like resolution, image size, etc.
3. affine: this component stores a transformation array that describes the relationship between data array and a reference space.

The concept of affine is described in detail here (https://nipy.org/nibabel/coordinate_systems.html), but basically it allows us to relate numbers in the data array (i, j, k) to a reference space, usually either a "real-world" scanner space or a standard template space (right-left, anterior-posterior, superior-inferior).

We can take a look at these components from the sample MRiShare data.

In [ ]:
# my_data_dir = '/data/ro_formateur/mrishare'
my_data_dir = '/data/ishare'
sample_T1 = op.join(my_data_dir, 'SHARE0001', 'anat', 'SHARE0001_T1w.nii.gz')

We use nibabel package to load the sample T1 image.

In [ ]:
import nibabel as nib

In [ ]:
sample_T1_img = nib.load(sample_T1)
type(sample_T1_img)

You can check the size of the image data directly:

In [ ]:
sample_T1_img.shape

You can explore other attributes of this nibabel image object using tab completion after "sample_T1_img".

In [ ]:
sample_T1_img.

Let's look at each component listed above.

First, we can access the data obj directly by img.dataobj or using method get_fdata().
The only difference between the two is how they occupy computational memory. You can read more about this here (https://nipy.org/nibabel/images_and_memory.html), and here (https://nipy.org/nibabel/nibabel_images.html#proxies-caching) but in most cases you can use one or the other to access data.

In [ ]:
sample_dataobj = sample_T1_img.dataobj
type(sample_dataobj)

In [ ]:
sample_dataobj.shape

In [ ]:
# value at approximate midpoint of the data
sample_dataobj[128, 128, 91]

In [ ]:
sample_data = sample_T1_img.get_fdata()
type(sample_data)

In [ ]:
sample_data.shape

In [ ]:
sample_data[128, 128, 91]

Let's try to take the mid-slice and look at it using plt.imshow.

In [ ]:
midslice_from_dataobj = sample_dataobj[:, :, 91]
plt.imshow(midslice_from_dataobj, cmap="gray")

Now let's look at the header.

In [ ]:
sample_T1_img.header

In [ ]:
print(sample_T1_img.header)

You can access individual entry of the heade information like in a Dictionary.

In [ ]:
sample_T1_img.header['dim']

Now let's look at the affine.

In [ ]:
sample_T1_img.affine

Since nibabel is not the only program that can read Nifti images, you can access similar information/metadata about the image using other commandline tools that comes with other software packages, like **fslhd** and **fslinfo** from FSL, **mri_info** from Freesurfer package.

In [ ]:
!fslinfo {sample_T1}

In [ ]:
!fslhd {sample_T1}

In [ ]:
!mri_info {sample_T1}

Likewise, many packages have specialized visualization tools to let you look at Nifti images. Try opening both T1 and FLAIR image in the data folder using the following viewers:

* FSLeyes: packaged with FSL
* freeview: packaged with Freesurfer
* MRIcron: standalone package


## Surface-based image formats

These are created when the volumetric image is processed to reconstruct a surface representation of the brain, as done by Freesurfer and other softwares. They are also essentially numerical arrays, but how this information is stored is different in different softwares. We will postpone examination of these formats until you learn about surface-based processing in the upcoming lectures.

But here is the good overview of various formats and how to read them in nibabel (https://nben.net/MRI-Geometry/#cortical-surfaces).
